In [1]:
from pyvis.network import Network
import networkx as nx

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from agents import generate_all_throws, powerset, SHEET_KEYS, YamsT1E

def roll_dice(n=5):
    return tuple(sorted(np.random.randint(1, 7, n)))


In [116]:
initial_throw = roll_dice()
initial_throw

(1, 1, 2, 4, 6)

In [117]:
lock_choices = powerset(initial_throw)
top_k = 5
n_nodes = 1 + len(lock_choices) * (top_k + 1) 
print(n_nodes)

net = Network()
agent = YamsT1E()
sheet = {k: None for k in SHEET_KEYS}


net.add_node("T1"+str(initial_throw), label=str(initial_throw), size=7, color='black')
nodes = []
lock_node_scores = {}
throw_node_scores = {}
for lock_choice in lock_choices:
    net.add_node("L1"+str(lock_choice), label=str(lock_choice), size=5, shape='square')
    net.add_edge("T1"+str(initial_throw), "L1"+str(lock_choice), value=1)
    
    random_throw_nodes = []
    for random_throw in generate_all_throws(lock_choice):
        random_throw_score = agent.score_throw(sheet, random_throw)
        random_throw_nodes.append(
            (
                "T2"+str(lock_choice)+str(random_throw), 
                random_throw_score,
                random_throw
            )
        )
        
    # topk_nodes = sorted(random_throw_nodes, key=lambda x: -x[1][1])[:top_k] # best throws
    # topk_nodes = sorted(random_throw_nodes)[:top_k] # sorted throws by dice values
    topk_idx = np.random.choice(len(random_throw_nodes), top_k, 
                                replace=len(random_throw_nodes)<top_k) 
    topk_nodes = [random_throw_nodes[i] for i in topk_idx]
    
    for (node, score, rnd_throw) in topk_nodes:
        net.add_node(node, label=str(rnd_throw), title=f'{score[0]}: {score[1]:.2f}', size=3)
        net.add_edge("L1"+str(lock_choice), node, value=1)
        throw_node_scores[node] = score[1]

    lock_score = np.mean([r[1][1] for r in random_throw_nodes])
    lock_node_scores["L1"+str(lock_choice)] = lock_score
    net.get_node("L1"+str(lock_choice))["title"] = str(f'{lock_score:.2f}')


145


In [118]:
from plotly.colors import sequential
from branca import colormap
from ast import literal_eval

throw_cmap = colormap.LinearColormap(
    [literal_eval(c.replace('rgb', '')) for c in sequential.Blues],
    vmin=min(throw_node_scores.values()),
    vmax=max(throw_node_scores.values()),
)
lock_cmap = colormap.LinearColormap(
    [literal_eval(c.replace('rgb', '')) for c in sequential.Greens],
    vmin=min(lock_node_scores.values()),
    vmax=max(lock_node_scores.values()),
)

In [119]:
for (node, score) in lock_node_scores.items():
    net.get_node(node)["color"] = lock_cmap(score)
for (node, score) in throw_node_scores.items():
    net.get_node(node)["color"] = throw_cmap(score)

In [120]:
for edge in net.get_edges():
    edge["color"] = net.get_node(edge["to"])["color"]


In [121]:
import os

filename = os.path.join(
    os.path.curdir,
    'data/YamsDecision.html'
)
net.show(filename, notebook=False)

.\data/YamsDecision.html
